In [1]:
# Задание №1.
from string import punctuation
from razdel import sentenize
from razdel import tokenize as razdel_tokenize
import numpy as np

In [2]:
# на основе данного корпуса будет работать языковая модель
news = open('lenta.txt', encoding="utf8").read()

In [3]:
# Шаг 1. Нормализуем текст
def normalize(text):
    normalized_text = [word.text.strip(punctuation) for word \
                                                            in razdel_tokenize(text)]
    normalized_text = [word.lower() for word in normalized_text if word and len(word) < 20 ]
    return normalized_text

In [4]:
norm_news = normalize(news)

In [5]:
# Шаг 2. Выведем частотность для отедльных слов
from collections import Counter

In [6]:
vocab_news = Counter(norm_news)

In [7]:
vocab_news.most_common(10) # абсолютные частоты 

[('в', 72412),
 ('и', 33290),
 ('на', 28434),
 ('по', 19490),
 ('что', 17031),
 ('с', 15921),
 ('не', 12702),
 ('из', 7727),
 ('о', 7515),
 ('как', 7514)]

In [8]:
# превращаем абсолютные частоты в вероятности
probas_news = Counter({word:c/len(norm_news) for word, c in vocab_news.items()})
probas_news.most_common(20)

[('в', 0.04808907489694771),
 ('и', 0.0221080111489724),
 ('на', 0.018883123731146926),
 ('по', 0.012943380513471676),
 ('что', 0.011310349590812525),
 ('с', 0.01057319451795703),
 ('не', 0.008435444806676101),
 ('из', 0.005131529052211166),
 ('о', 0.00499073907433246),
 ('как', 0.0049900749706632205),
 ('к', 0.00407161959610543),
 ('за', 0.0040125143695431435),
 ('россии', 0.0036751497055696383),
 ('для', 0.003325831175549828),
 ('его', 0.003260084912295149),
 ('он', 0.0031704309169478593),
 ('от', 0.003066830744546547),
 ('сообщает', 0.003050228152815567),
 ('а', 0.0029180715226369697),
 ('также', 0.002716184007188258)]

In [9]:
from nltk.tokenize import sent_tokenize

In [10]:
# Шаг 3. Находим количество вхождений для каждого н-грамма
def trigrammer(tokens, n=3):
    ngrams = []
    for i in range(0,len(tokens)-n+1):
        ngrams.append(' '.join(tokens[i:i+n]))
    return ngrams

def bigrammer(tokens, n=2):
    ngrams = []
    for i in range(0,len(tokens)-n+1):
        ngrams.append(' '.join(tokens[i:i +n]))
    return ngrams

In [11]:
# Шаг 4.Считаем вероятность 3-грамм. Для этого добавляем дополнительный тэг <start> и тэг окончания <end>
sentences_news = [['<start>'] + ['<start>'] + normalize(text) + ['<end>'] for text in sent_tokenize(news)]

In [12]:
unigrams = Counter()
bigrams = Counter()
trigrams = Counter()

for sentence in sentences_news:
    unigrams.update(sentence)
    bigrams.update(bigrammer(sentence)) 
    trigrams.update(trigrammer(sentence)) 

In [13]:
# Шаг 5. Создание матрицы 3-грамм
from scipy.sparse import lil_matrix, csr_matrix, csc_matrix

In [14]:
matrix_news = lil_matrix((len(bigrams), len(unigrams)))

id2bigram = list(bigrams)
bigram2id = {word:i for i, word in enumerate(id2bigram)}
id2unigram = list(unigrams)
unigram2id = {word:i for i, word in enumerate(id2unigram)}

for ngram in trigrams:
    word1, word2, word3 = ngram.split()
    bigram = ' '.join([word1, word2])
    matrix_news[bigram2id[bigram], unigram2id[word3]] = (trigrams[ngram]/
                                                                      bigrams[bigram])

In [15]:
# Шаг 6. Генерируем текст
def generate(matrix_news, id2unigram, bigram2id, n=100, start = '<start> <start>'):
    text = []  
    current_bigram = start 
    
    for i in range(n): 
        bigram_id = bigram2id[current_bigram]  
        chosen_word_idx = np.random.choice(matrix_news.shape[1], p=matrix_news[bigram_id].toarray()[0]) 
        word = id2unigram[chosen_word_idx]  
        text.append(word)
        current_bigram = current_bigram.split()[1] + ' ' + word
        
        if word == '<end>': 
            current_bigram = '<start> <start>'         
    return ' '.join(text)

In [16]:
print(generate(matrix_news, id2unigram, bigram2id).replace('<end>', '\n'))

представители network solutions согласилась также на ряде избирательных участков в этих акциях 
 напомним что ходе заседания правительства не обсуждать тему формирования кабинета министров на котором обсуждаются приоритетные направления по обеспечению прав и объявил голодовку в знак протеста против действующих властей 
 сегодня от 10 до 25 миллионов долларов 
 координатор проекта марина литвинович фонд эффективной политики была доставлена в манхэттенский окружной суд американского штата аризона с просьбой отменить распоряжение об обязательных поставках нефти на заводы внутри страны и преодоление продовольственного кризиса заявил риа новости со ссылкой на заместителя министра финансов лешека бальцеровича 
 начало забастовки намечалось на 28 процентов


In [17]:
print(generate(matrix_news, id2unigram, bigram2id).replace('<end>', '\n'))

к поискам кота подключен интерпол в пятницу интерфаксу лидер думской фракции единство 
 еще 16 по контракту 
 пожар силами нарядов вызванных к месту взрыва прибыли скорая помощь и по сути переговорный процесс ни о личности раненого 
 у россии авианесущего крейсера адмирал горшков и модернизации флота 
 во втором туре выборов потерпел поражение 
 как сообщили риа новости двое из них 150 миллионов фунтов стерлингов а во втором круге проходящего в голландском городе энсхеде 
 ранее в проекте еще одной темой переговоров 
 березовский заявил что россия вместе с тем до конца основного времени тертышный сократил разрыв в счете


In [18]:
print(generate(matrix_news, id2unigram, bigram2id).replace('<end>', '\n'))

он напомнил что срок полномочий кармокова он готовит почву заигрывая с коммунистами содержится в национальной валюте 
 по данным мониторинговых фирм на сегодня долг коммунальных служб поэтому в нашу пользу 
 все удостоверения подлинные имеют подписи руководителей гувд и регистрационной палатой города об утерянных и похищенных паспортах 
 инцидент произошел 19 июня в большом колонном зале дома союзов состояласьцеремония инаугурации губернатора подмосковья бориса громова ивице-губернатора михаила меня 
 но именно после этой встречи и принес ответ 
 поводом для этого шесть из семи пунктов 
 по словам американского военного ведомства взрыв произошел в 15 километрах от населенного пункта улус-керт ликвидированы


In [19]:
# Шаг 7. Считаем перплексию
def perplexity(logp, N):
    return np.exp((-1/N) * logp)

In [20]:
def compute_join_trigram(news, word_counts, bigram_counts, trigram_counts):
    prob = 0
    tokens = normalize(news)
    for ngram in trigrammer(['<start>', '<start>'] + tokens + ['<end>']):
        word1, word2, word3 = ngram.split()
        bigram = ' '.join([word1, word2])
        if word1 in word_counts and ngram in trigram_counts:
            prob += np.log(trigram_counts[ngram]/bigram_counts[bigram])
        else:
            prob += np.log(2e-5)
    
    return prob, len(tokens)

In [21]:
ps_trigram = []
for sent in sent_tokenize(news[:50]): # отложенные предложения
    prob, N = compute_join_trigram(sent, unigrams, bigrams, trigrams)
    if not N:
        continue
    ps_trigram.append(perplexity(prob, N))

In [22]:
np.mean(ps_trigram) # перплексия для всей модели

109.67360810071587

In [23]:
phrase = 'Основной задачей окружных органов юстиции станет формирование единого правового пространства.'
log_prob, N = compute_join_trigram(phrase, unigrams, bigrams, trigrams)
perplexity(log_prob, N) #для одного предложения

4.320690281132484